In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import glob
import os
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import nivapy

In [2]:
# Connect to db
ora_eng = nivapy.da.connect(src='nivabase')

Username: ········
Password: ········
Connection successful.


In [3]:
# Connect to db
pg_eng = nivapy.da.connect(src='postgres')

Username: ········
Password: ········
Connection successful.


# Critical Loads Workflow - new grid

This notebook implements the same workflow as the one [here](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow.ipynb), but using the new 0.1 degree vector grid instead of the old BLR grid. Note that this requires adopting an entirely raster-based workflow, so the procedure for water and soils critical loads is different to that described in the previous notebook. The vegetation calculations were already raster-based, so the methodology used here is similar.

Details of the spatial processing required to enable switching to the new 0.1 degree deposition grid are provided in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/new_grid.ipynb).

**NB:** This notebook is designed for processing one dataset at a time i.e. the looping over time periods implemented in the previous notebook is not repeated here (the previous notebook was used for historic calculations and evaluating the new code base, which is not applicable here).

## 1. Vegetation

A new raster-based workflow was developed in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_vegetation.ipynb). The code below creates vegetation exceedance grids (in mg-N/l) for the time periods of interest.

### 1.1. Upload new data to database

The new deposition data from NILU is supplied as `.dat` files. For an example, see:

C:\Data\James_Work\Staff\Kari_A\Critical_Loads\raw_data\2012_2016

In [4]:
# ID for new series
ser_id = 28

# Create new series for this data period
df = pd.DataFrame({'dep_series_id':ser_id,
                   'name':'Middel 2012-2016 (new; hi-res)',
                   'description':'Fordelt til BLR av NILU 2017 (Wenche Aas; new method; 0.1 degree grid)'},
                  index=[0,])

## Add to db
#df.to_sql('dep_series_definitions', con=ora_eng, schema='resa2', 
#          if_exists='append', index=False)

In [5]:
# Read NILU data
data_fold = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\raw_data\2012_2016'
search_path = os.path.join(data_fold, '*.dat')
file_list = glob.glob(search_path)

df_list = []
for fpath in file_list:
    # Get par name
    name = os.path.split(fpath)[1].split('_')[:2]
    name = '_'.join(name)
    
    # Read file
    df = pd.read_csv(fpath, delim_whitespace=True, header=None,
                     names=['lat', 'lon', name])
    df.set_index(['lat', 'lon'], inplace=True)    
    df_list.append(df)

# Combine
df = pd.concat(df_list, axis=1)
df.reset_index(inplace=True)

# Calculate unique integer cell ID as latlon 
# (both *100 and padded to 4 digits)
df['cell_id'] = ((df['lat']*100).astype(int).map('{:04d}'.format) + 
                 (df['lon']*100).astype(int).map('{:04d}'.format))
df['cell_id'] = df['cell_id'].astype(int)
del df['lat'], df['lon'], df['tot_n']

# Rename
df.rename(columns={'tot_nhx':2, # N (red)
                   'tot_nox':1, # N (oks)
                   'tot_s':4},  # Non-marine S
          inplace=True)

# Melt 
df = pd.melt(df, var_name='parameter_id', id_vars='cell_id')

# Add series ID
df['dep_series_id'] = ser_id

## Add to db
#df.to_sql('dep_grid_0_1deg_values', con=ora_eng, schema='resa2', 
#          if_exists='append', index=False)

df.head()

,cell_id,parameter_id,value,dep_series_id
0,50050305,2,270.87,28
1,50050315,2,240.50,28
2,50050325,2,259.19,28
3,50050335,2,252.06,28
4,50050345,2,332.82,28


### 1.2. Read lookup table linking vegetation classes to critical loads

In [6]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

,CL_100smgN/m2/yr
NORUTcode,
1,5
2,5
3,5
4,10
5,10


### 1.3. Reclassify

Unless the critical loads change, this cell only needs running once. `sat_veg_30m_cr_lds_div100.tif` can then be used in subsequent calculations.

In [7]:
## Input national veg map
#in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#          r'\GIS\Raster\vegetation\sat_veg_30m_all.tif')
#
## Output geotiff for critical loads values
#out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#           r'\GIS\Raster\vegetation\sat_veg_30m_cr_lds_div100.tif')
#
## Mask raster for land defined by BLR
#mask_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
#            r'\GIS\Raster\blr_land_mask.tif')
#
## Reclassify
#cl.reclassify_raster(in_tif, mask_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`. The resulting dataframe should have just a single column.

In [8]:
# Get all dep values
sql = ("SELECT cell_id, dep_series_id as series, value as dep "
       "FROM resa2.dep_grid_0_1deg_values "
       "WHERE parameter_id IN (1, 2) "
       "AND dep_series_id = 28")

df = pd.read_sql(sql, ora_eng)

# Sum N_oks and N_red
df = df.groupby(['cell_id', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
df.columns = ['Ndep1216_3'] # '3' means 'new method; new grid'. Must be <=10 chars for shp

print np.nanmin(df.values), np.nanmax(df.values)

df.head()

6.32 3593.19


,Ndep1216_3
cell_id,
50050305,497.23
50050315,455.25
50050325,473.62
50050335,471.73
50050345,574.69


### 1.5. Process vegetation data

In [9]:
%%time

# Path to raw BLR shapefile
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\grid_0_1deg\crit_lds_0_1deg_grid_clip_utm_z33n.shp')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\vegetation\sat_veg_30m_all.tif')

# Critical loads raster
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\vegetation\sat_veg_30m_cr_lds_div100.tif')

# Container for output
data_dict = {'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in df.columns:    
    
    print 'Processing: %s' % ser
    print '    Building deposition shapefile...'
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['cell_id', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='cell_id')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
               r'\Shapefiles\deposition\dep_%s.shp' % ser)
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print '    Rasterising shapefile...'
    
    # Output BLR raster
    dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
               r'\GIS\Raster\deposition\dep_%s_30m.tif' % ser)

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print '    Calculating exceedance...'
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print '    Saving exceedance grid...'
    ex_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
              r'\GIS\Raster\exceedance\exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    del ex_grid
    print '    Done.'

# Build output df
ex_df = pd.DataFrame(data_dict, index=['vegetation',])
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df = ex_df.round(0).astype(int)

print 'Finished.'

Processing: Ndep1216_3
    Building deposition shapefile...
    Rasterising shapefile...


C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:77: RuntimeWarning: invalid value encountered in greater
C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:80: RuntimeWarning: invalid value encountered in less


    Calculating exceedance...
    Saving exceedance grid...
    Done.
Finished.
Wall time: 2min 24s


### 1.6. Print national exceedance summary for vegetation

In [10]:
# Exceedance summary
ex_df

,ex_area_km2,nor_area_km2,ex_pct
vegetation,64264,322184,20


## 2. Water

The calculations for soil and water involve parameters estimated on the old BLR grid. We do not want/are not able to recalculate these values, so instead I'll use a raster-based workflow where the deposition (0.1 degree grid) and parameter (BLR grid) values are all converted to a standard 120 m grid, and then processed in a consistent way. 

### 2.1. Deposition data

#### 2.1.1. Extract deposition data

**Note:** Remember to add the new `dep_series_id` to the query below.

This data is on a 0.1 degree grid.

In [11]:
# Get dep data
sql = ("SELECT a.cell_id, b.name AS series, c.name AS par, a.value AS dep "
       "FROM resa2.dep_grid_0_1deg_values a, "
       "resa2.dep_series_definitions b, "
       "resa2.air_parameter_definitions c "
       "WHERE a.parameter_id IN (1, 2, 4) "
       "AND a.dep_series_id = 28 "
       "AND a.parameter_id = c.parameter_id "
       "AND a.dep_series_id = b.dep_series_id")

dep_df = pd.read_sql(sql, ora_eng)

# Tidy
dep_df['series'] = dep_df['series'].str[7:]
dep_df['par'] = dep_df['par'].str[:1]

# Group N(oks) and N(red)
dep_df = dep_df.groupby(['cell_id', 'series', 'par']).sum()

# Unstack
dep_df = dep_df.unstack('par')
dep_df.columns = dep_df.columns.get_level_values('par')

# Convert to meq/l
dep_df['N'] = dep_df['N'] / 14.
dep_df['S'] = dep_df['S']*2. / 32.06

# Unstack
dep_df = dep_df.unstack('series')

# Flatten col index
dep_df.columns = ['%s_%s' % (p, d) for p, d in zip(dep_df.columns.get_level_values('par'), 
                                                   dep_df.columns.get_level_values('series'))]
dep_df.reset_index(inplace=True)

dep_df.head()

,cell_id,N_2012-2016 (new; hi-res),S_2012-2016 (new; hi-res)
0,50050305,35.516429,7.398004
1,50050315,32.517857,7.159077
2,50050325,33.830000,7.618840
3,50050335,33.695000,6.792265
4,50050345,41.049286,7.900187


#### 2.1.2. Convert to raster

The code below gets the 0.1 degree spatial data from the database and creates a shapefile including the deposition data above. This is then converted to two 120 m rasters (one for N and one for S).

In [12]:
# Read 0.1 deg spatial data
sql = ("SELECT * FROM public.dep_grid_0_1deg")
crs = {'init': 'epsg:4326'} # WGS84
gdf = gpd.GeoDataFrame.from_postgis(sql, geom_col='geom', crs=crs, con=pg_eng)

# Project 
gdf = gdf.to_crs({'init': 'epsg:32633'}) # UTM Zone 33N

# Join
gdf = gdf.merge(dep_df, on='cell_id')

# Tidy
del gdf['lat'], gdf['lon'], gdf['area_m2'], gdf['cell_id']

# Simplify col names due to shp limits
gdf.columns = [i.split('_')[0] for i in gdf.columns]

# Save to shp
dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
           r'\deposition\dep_ns_meq_2012_16_new_meth_0_1deg.shp')
gdf.to_file(driver='ESRI Shapefile', filename=dep_shp)

gdf.head()

,geom,N,S
0,(POLYGON ((71329.03767688642 6901331.461400039...,31.942857,7.791641
1,"(POLYGON ((566881.4284799806 7543730.21153602,...",14.319286,10.567062
2,(POLYGON ((3599.541798072867 6718792.067247962...,55.578571,13.972551
3,(POLYGON ((11158.06838512811 6605546.052893229...,63.875000,14.380536
4,"(POLYGON ((1020339.017845268 7801865.54714183,...",6.342143,5.036182


In [13]:
# 120 m snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
            r'\Raster\vegetation\sat_veg_120m_all.tif')

# Raster folder
data_fold = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Raster'

# Loop over pars
for col in ['N', 'S']:
    # Output raster    
    dep_tif = os.path.join(data_fold, 'deposition', 'dep_2012_16_%s_meq_120m.tif' % col)

    # Rasterise shapefile
    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, col, -1, gdal.GDT_Float32)

### 2.2. Critical loads

#### 2.2.1. Get critical loads data

In [14]:
# Get dep data
sql = ("SELECT blr, claoaavaroaa, eno3fl, clminn, clmaxnoaa, clmaxsoaa "
       "FROM resa2.cla")
cl_df = pd.read_sql(sql, ora_eng)

# Set index
cl_df.index = cl_df['blr']
del cl_df['blr']

# Add CLSmin as 0
cl_df['clmins'] = 0

# Tidy
cl_df.reset_index(inplace=True)

cl_df.head()

,blr,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
0,60506013,46.689055,20.499,3.204124,59.153507,47.083568,0
1,60508003,47.240467,8.988,3.204124,70.544508,48.610725,0
2,60509002,48.396218,0.029,14.635616,100.955383,51.368535,0
3,60511002,11.156834,0.080,7.077605,27.656760,11.903564,0
4,60511006,34.764804,1.081,42.316831,121.992502,38.498971,0


#### 2.2.2. Convert to raster

In [15]:
# Read BLR shp
blr_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
           r'\BLR\blrgrid_uten_grums_utm_z33n.shp')
gdf = gpd.read_file(blr_shp)
gdf['blr'] = gdf['BLR']
gdf = gdf[['geometry', 'blr']]

# Join
gdf = gdf.merge(cl_df, on='blr')

# Tidy
del gdf['blr']

# Save to shp
cl_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
          r'\water_cl_pars_blr_grid.shp')
gdf.to_file(driver='ESRI Shapefile', filename=cl_shp)

# Loop over cols
cols = [i[:10] for i in gdf.columns if i != 'geometry'] # 10 char limit in shp files
for col in cols:
    # Output raster
    cl_tif = os.path.join(data_fold, '%s_120m.tif' % col)

    # Rasterise shapefile
    cl.vec_to_ras(cl_shp, cl_tif, snap_tif, col, -1, gdal.GDT_Float32)    

gdf.head()

,geometry,claoaavaroaa,eno3fl,clminn,clmaxnoaa,clmaxsoaa,clmins
0,POLYGON ((-10886.67732153146 6503779.108531611...,28.533538,35.544,3.204124,40.514435,29.056767,0
1,"POLYGON ((3660.479678658361 6501900.217185441,...",32.029772,27.861,3.204124,43.473268,32.471152,0
2,"POLYGON ((18211.98529702786 6500076.037824233,...",20.888186,25.137,3.204124,29.564269,21.184960,0
3,"POLYGON ((32767.70791637653 6498306.548390091,...",18.914393,36.846,25.972633,49.911208,19.189395,0
4,POLYGON ((-11562.15574961301 6498623.842200487...,20.463044,47.587,3.204124,30.057701,20.846989,0


### 2.3. Calculate exceedances

#### 2.3.1. SSWC model

In [16]:
# Read grids
tif_path = os.path.join(data_fold, 'deposition', 'dep_2012_16_S_meq_120m.tif')
s_dep, s_ndv = cl.read_geotiff(tif_path)

tif_path = os.path.join(data_fold, 'eno3fl_120m.tif')
eno3fl, eno_ndv = cl.read_geotiff(tif_path)

tif_path = os.path.join(data_fold, 'claoaavaro_120m.tif')
claoaavaroaa, cla_ndv = cl.read_geotiff(tif_path)

# Set ndv
s_dep[s_dep==s_ndv] = np.nan
eno3fl[eno3fl==eno_ndv] = np.nan
claoaavaroaa[claoaavaroaa==cla_ndv] = np.nan

# Get total area of non-NaN from dep grid
nor_area = np.count_nonzero(~np.isnan(s_dep))*120.*120./1.E6

# Exceedance
sswc_ex = s_dep + eno3fl - claoaavaroaa
del s_dep, eno3fl, claoaavaroaa  

# Get total area exceeded
ex_area = np.count_nonzero(sswc_ex > 0)*120.*120./1.E6

# Set <0 to 0
sswc_ex[sswc_ex<0] = 0

# Write geotif
sswc_tif = os.path.join(data_fold, 'exceedance', 'sswc_exceed_12_16_meq.tif')
cl.write_geotiff(sswc_ex, sswc_tif, snap_tif, -1, gdal.GDT_Float32)
del sswc_ex

# Build df
ex_pct = 100 * ex_area / nor_area
ex_df = ex_df.append(pd.DataFrame({'ex_area_km2':ex_area,
                                   'nor_area_km2':nor_area,
                                   'ex_pct':ex_pct},
                                  index=['water_sswc',]))
ex_df = ex_df.round(0).astype(int)

ex_df

C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:24: RuntimeWarning: invalid value encountered in greater
C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:27: RuntimeWarning: invalid value encountered in less


,ex_area_km2,ex_pct,nor_area_km2
vegetation,64264,20,322184
water_sswc,23298,7,322184


#### 2.3.3. FAB model

The processing for the FAB model is quite complicated, because of the nature of the calculations required to estimate exceedances. My [original workflow](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow.ipynb) using the BLR grid tabulated values for the parameters of interest in each BLR cell (`cln_min, cln_max, cls_min, cls_max, dep_n and dep_s`) and then looped over the rows in this dataframe using the function `exceed_ns_icpm()` (see section 2.3.3 of [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_workflow.ipynb) for details).

Looping over dataframes is slow, but for just a few thousand BLR cells it only takes a few seconds. However, in order to combine data from the old BLR grid with the new deposition grid, all calculations are now being rasterised at 120 m resolution. For a bounding box covering the whole of Norway, this correpsonds to a grid of $10349 \times 14122 = 146 148 578$ cells in total. Looping over 150 million cells using Pandas is not a feasible approach (it's likely to takes days to weeks).

The best solution is to construct a "vectorised" algorithm that works directly with the 2D arrays (i.e. no looping required). Vectorising the logic required by the FAB calculations is quite a challenge, but I think I've managed it - see `vectorised_exceed_ns_icpm()` for details.

In [17]:
# Read CL arrays
array_dict = {}
for name in ['clminn', 'clmaxnoaa', 'clmins', 'clmaxsoaa']:
    # Read tif
    tif_path = os.path.join(data_fold, '%s_120m.tif' % name)
    data, ndv = cl.read_geotiff(tif_path)
    
    # Set NDV
    data[data==ndv] = np.nan
    
    # Add to dict
    array_dict[name] = data
    
# Read dep arrays
for name in ['dep_2012_16_N_meq_120m', 'dep_2012_16_S_meq_120m']:
    # Read tif
    tif_path = os.path.join(data_fold, 'deposition', '%s.tif' % name) 
    data, ndv = cl.read_geotiff(tif_path)
    
    # Set NDV
    data[data==ndv] = np.nan
    
    # Add to dict
    array_dict[name] = data

In [18]:
%%time
# Extract arrays from dict
cln_min = array_dict['clminn']
cln_max = array_dict['clmaxnoaa']
cls_min = array_dict['clmins']
cls_max = array_dict['clmaxsoaa']
dep_n = array_dict['dep_2012_16_N_meq_120m']
dep_s = array_dict['dep_2012_16_S_meq_120m']

# Estimate exceedances
ex_n, ex_s, reg_id = cl.vectorised_exceed_ns_icpm(cln_min, cln_max, 
                                                  cls_min, cls_max, 
                                                  dep_n, dep_s)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0)*120.*120./1.E6
nor_area = np.count_nonzero(~np.isnan(dep_s))*120.*120./1.E6
ex_pct = 100*ex_area/nor_area

ex_df = ex_df.append(pd.DataFrame({'ex_area_km2':ex_area,
                                   'nor_area_km2':nor_area,
                                   'ex_pct':ex_pct},
                                  index=['water_fab',]))
ex_df = ex_df.round(0).astype(int)

critical_loads.py:302: RuntimeWarning: invalid value encountered in less
  mask = ((cln_min < 0) | (cln_max < 0) | (cls_min < 0) | (cls_max < 0))
critical_loads.py:326: RuntimeWarning: invalid value encountered in less_equal
  ((dep_n - cln_max)*ds <= (dep_s - cls_min)*dn) &
critical_loads.py:334: RuntimeWarning: invalid value encountered in less_equal
  mask = ((dep_s <= cls_min) & (edited == 0))
critical_loads.py:341: RuntimeWarning: invalid value encountered in less_equal
  mask = ((dep_n <= cln_min) & (edited == 0))
critical_loads.py:348: RuntimeWarning: invalid value encountered in greater_equal
  mask = ((-(dep_n - cln_max)*dn >= (dep_s - cls_min)*ds) & (edited == 0))
critical_loads.py:355: RuntimeWarning: invalid value encountered in less_equal
  mask = ((-(dep_n - cln_min)*dn <= (dep_s - cls_max)*ds) & (edited == 0))
critical_loads.py:365: RuntimeWarning: invalid value encountered in divide
  xf = (dn*s + ds*v)/dd
critical_loads.py:366: RuntimeWarning: invalid value encountered

Wall time: 21.6 s


C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: RuntimeWarning: invalid value encountered in greater


In [19]:
# Write results to geotif
# N
tif_path = os.path.join(data_fold, 'exceedance', 'fab_exceed_n_12_16_meq.tif')
cl.write_geotiff(ex_n, tif_path, snap_tif, -1, gdal.GDT_Float32)

# S
tif_path = os.path.join(data_fold, 'exceedance', 'fab_exceed_s_12_16_meq.tif')
cl.write_geotiff(ex_s, tif_path, snap_tif, -1, gdal.GDT_Float32)

# N+S
tif_path = os.path.join(data_fold, 'exceedance', 'fab_exceed_ns_12_16_meq.tif')
cl.write_geotiff(ex_n+ex_s, tif_path, snap_tif, -1, gdal.GDT_Float32)

# Exceedance 'region'
tif_path = os.path.join(data_fold, 'exceedance', 'fab_reg_id.tif')
cl.write_geotiff(reg_id, tif_path, snap_tif, -1, gdal.GDT_Float32)

In [20]:
ex_df

,ex_area_km2,ex_pct,nor_area_km2
vegetation,64264,20,322184
water_sswc,23298,7,322184
water_fab,61080,19,322184


##### Aside: Testing against "looping" approach

The results above look reasonable, but the vectorised code is complicated and I'd like to check it's giving the correct results. The code below implements looping, but this time using a more optimised approach compared to Pandas. As [I have found previously](http://nbviewer.jupyter.org/github/JamesSample/misc/blob/master/radb.ipynb#2.-NCBI-dataset), when looping is unavoidable, dropping down to basic Numpy from Pandas can give speed-ups of roughly 1000x, and further gains can be made by using Numba and the @jit decorator. 

In [21]:
%%time
from numba import jit

@jit
def fab_loop_numpy(array_dict):
    # Get desried shape
    shp = array_dict['clminn'].shape
    
    # Ravel
    for key in array_dict.keys():
        array_dict[key] = array_dict[key].ravel()
        
    # Extract ravelled arrays
    cln_min = array_dict['clminn']
    cln_max = array_dict['clmaxnoaa']
    cls_min = array_dict['clmins']
    cls_max = array_dict['clmaxsoaa']
    dep_n = array_dict['dep_2012_16_N_meq_120m']
    dep_s = array_dict['dep_2012_16_S_meq_120m']   
    
    # Pre-allocate result arrays
    ex_n_ar = np.full(shape=dep_n.shape, fill_value=np.nan)
    ex_s_ar = np.full(shape=dep_n.shape, fill_value=np.nan)
    reg_id_ar = np.full(shape=dep_n.shape, fill_value=np.nan)
    
    for idx in xrange(len(dep_n)):
        # Test for NaN
        if np.isnan(dep_s[idx]) or np.isnan(cln_min[idx]):
            pass
        
        else:
            # Calc exceedance
            ex_n, ex_s, reg_id = cl.exceed_ns_icpm(cln_min[idx], cln_max[idx],
                                                   cls_min[idx], cls_max[idx],
                                                   dep_n[idx], dep_s[idx])
            ex_n_ar[idx] = ex_n
            ex_s_ar[idx] = ex_s
            reg_id_ar[idx] = reg_id  
    
    # Reshape
    ex_n_ar = ex_n_ar.reshape(shp)
    ex_s_ar = ex_s_ar.reshape(shp)
    reg_id_ar = reg_id_ar.reshape(shp)
    
    return (ex_n_ar, ex_s_ar, reg_id_ar)

# Run looped version
ex_n2, ex_s2, reg_id2 = fab_loop_numpy(array_dict)

# Get exceeded area
ex = ex_n + ex_s
ex_area = np.count_nonzero(ex > 0)*120.*120./1.E6
nor_area = np.count_nonzero(~np.isnan(ex))*120.*120./1.E6
ex_pct = 100*ex_area/nor_area

C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:51: RuntimeWarning: invalid value encountered in greater


Wall time: 4min 18s


In [22]:
# Print results for comparison
pd.DataFrame({'ex_area_km2':ex_area,
              'nor_area_km2':nor_area,
              'ex_pct':ex_pct},
             index=['water_fab',]).round(0).astype(int)

,ex_area_km2,ex_pct,nor_area_km2
water_fab,61080,19,320584


The code above produces the same output as the vectorised version, but it's around 10 times slower. Based on this, I'm fairly happy that the vectorised version is producing consistent results.

## 3. Soil

The soil calculations use the "old" method - see e-mail from Kari received 01/11/2017 at 13.47. The equation is simply

$$E_{soil} = S_{dep} - CL_{soil}$$

### 3.1. Deposition data

#### 3.1.1. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [23]:
# Get all dep values for non-marine S
# in mg-S/m2/year
sql = ("SELECT cell_id, dep_series_id as series, value as dep "
       "FROM resa2.dep_grid_0_1deg_values "
       "WHERE parameter_id = 4 "
       "AND dep_series_id = 28")

dep_df = pd.read_sql(sql, ora_eng)

# Reshape and tidy
dep_df = dep_df.pivot(index='cell_id', columns='series', values='dep')
dep_df.columns.name = ''
dep_df.columns = ['Sdep1216_3'] # '3' means 'new method; new grid'. Must be <=10 chars for shp
dep_df.reset_index(inplace=True)

dep_df.head()

,cell_id,Sdep1216_3
0,50050305,118.59
1,50050315,114.76
2,50050325,122.13
3,50050335,108.88
4,50050345,126.64


#### 3.1.2. Rasterise

In [24]:
# Read 0.1 deg spatial data
sql = ("SELECT * FROM public.dep_grid_0_1deg")
crs = {'init': 'epsg:4326'} # WGS84
gdf = gpd.GeoDataFrame.from_postgis(sql, geom_col='geom', crs=crs, con=pg_eng)

# Project 
gdf = gdf.to_crs({'init': 'epsg:32633'}) # UTM Zone 33N

# Join
gdf = gdf.merge(dep_df, on='cell_id')

# Tidy
del gdf['lat'], gdf['lon'], gdf['area_m2'], gdf['cell_id']

# Simplify col names due to shp limits
gdf.columns = [i.split('_')[0] for i in gdf.columns]

# Save to shp
dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
           r'\deposition\dep_s_mg_2012_16_new_meth_0_1deg.shp')
gdf.to_file(driver='ESRI Shapefile', filename=dep_shp)

gdf.head()

,geom,Sdep1216
0,(POLYGON ((71329.03767688642 6901331.461400039...,124.90
1,"(POLYGON ((566881.4284799806 7543730.21153602,...",169.39
2,(POLYGON ((3599.541798072867 6718792.067247962...,223.98
3,(POLYGON ((11158.06838512811 6605546.052893229...,230.52
4,"(POLYGON ((1020339.017845268 7801865.54714183,...",80.73


In [25]:
# Output raster    
dep_tif = os.path.join(data_fold, 'deposition', 'dep_2012_16_s_mg_120m.tif')

# Rasterise shapefile
cl.vec_to_ras(dep_shp, dep_tif, snap_tif, 'Sdep1216', -1, gdal.GDT_Float32)

### 3.2. Critical loads

**Note:** Parameter 86 (`CL-Soil-N`) is strangely named, but the description implies it is the critical load for S.

**Note 2:** There is one cell where the critical load is negative. This is filtered out from the calculations - see e-mail from Espen received 16/11/2017 at 13.04.

#### 3.2.1. Get CL data

In [26]:
# Get all CLs for S
# in meq/m2/year
sql = ("SELECT blr, xvalue as crit_ld "
       "FROM resa2.talegren_values "
       "WHERE talegren_paramid = 86")

cl_df = pd.read_sql(sql, ora_eng)
cl_df.index = cl_df['blr']
del cl_df['blr']

# Convert to mg-S/m2/yr
cl_df['crit_ld'] = cl_df['crit_ld']*32.06 / 2.

# Remove negative CL
cl_df = cl_df.query('crit_ld >= 0')
cl_df.reset_index(inplace=True)

cl_df.head()

,blr,crit_ld
0,58006004,1818.923277
1,58006007,3237.448624
2,58006008,3095.109867
3,58006012,3443.772948
4,58006015,3062.583133


#### 3.2.2. Rasterise

In [27]:
# Read BLR shp
blr_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
           r'\BLR\blrgrid_uten_grums_utm_z33n.shp')
gdf = gpd.read_file(blr_shp)
gdf['blr'] = gdf['BLR']
gdf = gdf[['geometry', 'blr']]

# Join
gdf = gdf.merge(cl_df, on='blr')

# Tidy
del gdf['blr']

# Save to shp
cl_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS\Shapefiles'
          r'\soil_cl_pars_blr_grid.shp')
gdf.to_file(driver='ESRI Shapefile', filename=cl_shp)

# Output raster
cl_tif = os.path.join(data_fold, 'soil_cl_mg_120m.tif')

# Rasterise shapefile
cl.vec_to_ras(cl_shp, cl_tif, snap_tif, 'crit_ld', -1, gdal.GDT_Float32)    

gdf.head()

,geometry,crit_ld
0,"POLYGON ((32767.70791637653 6498306.548390091,...",1818.923277
1,"POLYGON ((16502.92229014495 6486223.97185181, ...",3237.448624
2,"POLYGON ((31110.65535350452 6484450.549406871,...",3095.109867
3,(POLYGON ((29455.80772649084 6470594.295637381...,3443.772948
4,(POLYGON ((9788.008991688606 6465964.999445073...,3062.583133


### 3.3. Calculate exceedances

In [28]:
# Read tifs
tif_path = os.path.join(data_fold, 'deposition', 'dep_2012_16_s_mg_120m.tif')
dep_s, ndv = cl.read_geotiff(tif_path)
dep_s[dep_s==ndv] = np.nan

# Read tifs
tif_path = os.path.join(data_fold, 'soil_cl_mg_120m.tif')
cl_soil, ndv = cl.read_geotiff(tif_path)
cl_soil[cl_soil==ndv] = np.nan

# Exceedance
ex_soil = dep_s - cl_soil
ex_soil[ex_soil<0] = 0

# Save geotiff
tif_path = os.path.join(data_fold, 'exceedance', 'soil_exceed_12_16_mg.tif')
cl.write_geotiff(ex_soil, tif_path, snap_tif, -1, gdal.GDT_Float32)

# Get exceeded area
ex_area = np.count_nonzero(ex_soil > 0)*120.*120./1.E6
nor_area = np.count_nonzero(~np.isnan(ex_soil))*120.*120./1.E6
ex_pct = 100*ex_area/nor_area

ex_df = ex_df.append(pd.DataFrame({'ex_area_km2':ex_area,
                                   'nor_area_km2':nor_area,
                                   'ex_pct':ex_pct},
                                  index=['soil',]))
ex_df = ex_df.round(0).astype(int)

ex_df

C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:13: RuntimeWarning: invalid value encountered in less
C:\Data\Anaconda2\lib\site-packages\ipykernel\__main__.py:20: RuntimeWarning: invalid value encountered in greater


,ex_area_km2,ex_pct,nor_area_km2
vegetation,64264,20,322184
water_sswc,23298,7,322184
water_fab,61080,19,322184
soil,0,0,108683


Note that `'nor_area_km2'` in the soils calculation actually refers to the total area with "productive forest" (see e-mail from Kari received 16/11/2017 at 12.52 for details), which is only 662 of the original BLR cells. If the soil exceedance is greater than zero, **the calculation of percent exceedance in the cell above will need modifying to use the true total area of Norway**. For now, it doesn't matter, as the exceeded area is zero.

In [29]:
# Write ex_df to file
out_csv = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\areas_exceeded_new_meth_new_grid.csv')
ex_df.to_csv(out_csv)